In [1]:
import numpy as np
import pandas as pd

seed = 3
np.random.seed(seed)

In [2]:
DATA_URL = "../../data_pipeline/notebooks/data/courses.csv"

courses_df = pd.read_csv(DATA_URL)
courses_df.head(1)

,CRN,Campus Description,Course Title,Subject Course,Faculty Name,Course Description,Term,Begin Time,End Time,Days,Prerequisites
0,34154,Online,Computer Science and Its Applications,CS1100,"Lieberherr, Karl",Introduces students to the field of computer s...,Spring 2025,NaN,NaN,NaN,[]


In [3]:
n_profs = 50

professors = courses_df["Faculty Name"].unique()
professors = np.random.choice(professors, size=n_profs, replace=False)

In [4]:
semester = "Spring 2025"

topics = [
    "Machine Learning",
    "Deep Learning",
    "Data Science",
    "Computer Systems",
    "Programming",
    "Development"
]

course_names = [
    'Foundations of Artificial Intelligence',
    'Algorithms',
    'Programming Design Paradigm',
    'Pattern Recognition and Computer Vision',
    'Natural Language Processing',
    'Deep Learning',
    'Machine Learning',
    'Information Retrieval',
    'Computer Systems',
    'Database Management',
    'Recitation for CS5010',
    'Foundations of Software Engineering',
    'Reinforcement Learning',
    'Computer/Human Interaction',
    'Web Development',
    'Compilers',
    'Mobile Application Development',
    'Discrete Structures',
    'Data Mining Techniques',
    'Fundamentals of Cloud Computing',
    'Object-Oriented Design',
    'Computer Systems'
]

In [5]:
seed_query_list = [
    "What courses are being offered for the {semester} semester?",
    "Which professor is taking the course {course_name}?",
    "What are the courses available related to {topic}?",
    "Can you please suggest online courses to take?",
    "Can you suggest research or seminar-based courses?",
    "Can you suggest courses without exams?",
    "Can you suggest courses which are not too hectic and easy to get good grades?",
    "What are the class timings for the course {course_name}?",
    "Are there any prerequisite courses for {course_name}?",
    "Has the course {course_name} been offered in the past? If so, could you summarize the reviews?",
    "How's the course {course_name} based on the reviews?",
    "How difficult is the course {course_name}?",
    "What courses are being offered by Professor {professor_name}?",
    "How is Professor {professor_name} in terms of grading?"
]

In [6]:
sft_dataset_creator_system_prompt = """
You are Qwen, a SFT Dataset creation tool. Given a question and context related to the question, you generate responses based on them.

Output Format:
Question:
{question}
Context:
{context}
Response:
{generated_response}
"""

prompt = "Context:\n{context}\n\nQuery: {query}\n\nAnswer:"

variant_system_prompt = """
You are Qwen, an LLM to help generate multiple variants of a question.

Given a question and the context, rephrase it in 10 different ways without altering it's meaning.
The rephrased question should have the same semantic meaning with different tone, style, etc.

Output Format:

The output should be in the following format.
{
    "question": [new_question_1, new_question_2, ..., new_question_10]
}
"""

In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
import torch
import gc

def free_gpu_memory():
    # Clear unused objects by forcing garbage collection
    gc.collect()

    # Empty the CUDA cache to release unused GPU memory
    torch.cuda.empty_cache()

    # Print current GPU memory usage for monitoring
    print(f"Current allocated GPU memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Current cached GPU memory: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

free_gpu_memory()

Current allocated GPU memory: 0.00 MB
Current cached GPU memory: 0.00 MB


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def generate_response(system_prompt, user_prompt, model_name="Qwen/Qwen2.5-3B-Instruct"):
    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare the message and text for generation
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize the input and move to appropriate device
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate the output
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )

    # Decode and return the response
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [10]:
# from time import time

# system_prompt = "You are Qwen, the useful LLM"
# prompt = "Give me a short introduction to large language model."

# start_time = time()
# response = generate_response(system_prompt, prompt)
# print(response)
# end_time = time()

# print(f"Took {end_time - start_time} secs.")

In [11]:
# questions = []

# for seed_query in seed_query_list:
#     if "professor_name" in seed_query:
#         for professor_name in professors:
#             new_seed_query = seed_query.format(professor_name=professor_name)
            
#             # Make multiple variants of the same question
#             variant_response = generate_response(variant_system_prompt, new_seed_query)
#             variant_response = json.loads(variant_response)["question"]
#             questions.extend(variant_response)

#     elif "semester" in seed_query:
#         new_seed_query = seed_query.format(semester=semester)

#         # Make multiple variants of the same question
#         variant_response = generate_response(variant_system_prompt, new_seed_query)
#         variant_response = json.loads(variant_response)["question"]
#         questions.extend(variant_response)

#     elif "course_name" in seed_query:
#         for course_name in course_names:
#             new_seed_query = seed_query.format(course_name=course_name)

#             # Make multiple variants of the same question
#             variant_response = generate_response(variant_system_prompt, new_seed_query)
#             variant_response = json.loads(variant_response)["question"]
#             questions.extend(variant_response)

#     elif "topic" in seed_query:
#         for topic in topics:
#             new_seed_query = seed_query.format(topic=topic)

#             # Make multiple variants of the same question
#             variant_response = generate_response(variant_system_prompt, new_seed_query)
#             variant_response = json.loads(variant_response)["question"]
#             questions.extend(variant_response)

#     else:
#         # Make multiple variants of the same question
#         variant_response = generate_response(variant_system_prompt, seed_query)
#         variant_response = json.loads(variant_response)["question"]
#         questions.extend(variant_response)

In [12]:
# import pickle

# with open('questions.pkl', 'wb') as file:
#     pickle.dump(questions, file)

In [13]:
import pickle

with open("questions.pkl", "rb") as file:
    questions = pickle.load(file)

len(questions)

2430

In [14]:
import weave

from curriculum_compass.naive_rag.utils import get_device
from curriculum_compass.naive_rag.utils import load_config
from curriculum_compass.naive_rag.utils import load_embedding_model
from curriculum_compass.naive_rag.utils import generate_llm_response
from curriculum_compass.naive_rag.utils import load_model_and_tokenizer
from curriculum_compass.naive_rag.utils import initialize_chromadb_client
from curriculum_compass.naive_rag.retriever_utils import load_course_data
from curriculum_compass.naive_rag.reranker import Reranker
from curriculum_compass.naive_rag.course_retriever import CourseRAGPipeline
from curriculum_compass.naive_rag.review_retriever import ReviewsRAGPipeline
from curriculum_compass.naive_rag.integrated_rag import IntegratedRAGPipeline

weave.init(project_name="SFT_Dataset_Creation")

config_path = "../../naive_rag/config.json"
config = load_config(config_path)
chromadb_client = initialize_chromadb_client("../../naive_rag/chromadb")

# ===== Initialize the re-ranker ===========
device = get_device()
print(f"Using device: {device}")
reranker = Reranker(config['reranker_model_name'],device)

# ===== Initialize the CourseRagPipeline ===========

course_data = load_course_data(config['course_data_path'])
course_rag = CourseRAGPipeline(reranker)
course_rag.intiliaze_course_search_system(course_data)

# ===== Initialize the NaiveReviewsRAGPipeline ===========

embedding_model = load_embedding_model(config['embedding_model_name'])
collection = chromadb_client.get_or_create_collection("naive_rag_embeddings")
review_rag = ReviewsRAGPipeline(embedding_model, collection,reranker)
    
# ===== Initialize the IntegratedRAGPipeline ===========
integrated_rag = IntegratedRAGPipeline(course_rag, review_rag,config,device)

/home/sampath.ki/emnlp/conda_env/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/sampath.ki/emnlp/conda_env/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/sampath.ki/emnlp/conda_env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/i

AttributeError: _ARRAY_API not found

SystemError: <built-in function __import__> returned a result with an exception set

Logged in as Weights & Biases user: s-kishore.
View Weave data at https://wandb.ai/s-kishore/course_rag_system/weave
Logged in as Weights & Biases user: s-kishore.
View Weave data at https://wandb.ai/s-kishore/sft_dataset_creation/weave
Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
random_questions = np.random.choice(questions, size=250, replace=False)

In [16]:
from tqdm import tqdm

contexts = []
responses = []

for query in tqdm(random_questions):
    context, response = integrated_rag(query,config['course_k'],config['review_k'],config['final_k'])
    
    contexts.append(context)
    responses.append(response)

  0%|          | 0/250 [00:00<?, ?it/s]

Processing query: What must I learn before I can effectively study Machine Learning?
Retrieving course information and reviews...
Processing query: What must I learn before I can effectively study Machine Learning?
Retrieving relevant course information...
Cross-encoder reranking...
Successfully reranked 10 documents
Retrieving


Add of existing embedding ID: 12338
Add of existing embedding ID: 12339
Add of existing embedding ID: 12340
Add of existing embedding ID: 12341
Add of existing embedding ID: 12342
Add of existing embedding ID: 12343
Add of existing embedding ID: 12344
Add of existing embedding ID: 12345
Add of existing embedding ID: 12346
Add of existing embedding ID: 12347
Add of existing embedding ID: 12348
Add of existing embedding ID: 12349
Add of existing embedding ID: 12350
Add of existing embedding ID: 12351
Add of existing embedding ID: 12352
Add of existing embedding ID: 12353
Add of existing embedding ID: 12354
Add of existing embedding ID: 12355
Add of existing embedding ID: 12356
Add of existing embedding ID: 12357
Add of existing embedding ID: 12358
Add of existing embedding ID: 12359
Add of existing embedding ID: 12360
Add of existing embedding ID: 12361
Add of existing embedding ID: 12362
Add of existing embedding ID: 12363
Add of existing embedding ID: 12364
Add of existing embedding ID

Reranking
Combining and reranking all documents...
Successfully reranked 10 combined documents
Generating integrated response...


  0%|          | 1/250 [00:06<26:22,  6.36s/it]

...

🍩 https://wandb.ai/s-kishore/sft_dataset_creation/r/call/01941515-bff6-7540-a514-c8e13f989a23
Processing query: Which educator is responsible for the Database Management seminar?
Retrieving course information and reviews...
Processing query: Which educator is responsible for the Database Management seminar?
Retrieving relevant course information...
Cross-encoder reranking...
Successfully reranked 10 documents
Retrieving
Reranking
Combining and reranking all documents...
Successfully reranked 10 combined documents
Generating integrated response...


100%|█████████▉| 249/250 [26:57<00:06,  6.64s/it]

🍩 https://wandb.ai/s-kishore/sft_dataset_creation/r/call/01941515-e260-7b53-bb65-84f15175e32d
Processing query: Could you share the list of courses Professor Rachlin, John is delivering?
Retrieving course information and reviews...
Processing query: Could you share the list of courses Professor Rachlin, John is delivering?
Retrieving relevant course information...
Cross-encoder reranking...
Successfully reranked 10 documents
Retrieving
Reranking
Combining and reranking all documents...
Successfully reranked 10 combined documents
Generating integrated response...


100%|██████████| 250/250 [27:03<00:00,  6.49s/it]

🍩 https://wandb.ai/s-kishore/sft_dataset_creation/r/call/01941515-f78f-7740-a013-d7dc4a4d0e97


In [18]:
sft_df = pd.DataFrame()
sft_df["question"] = random_questions
sft_df["context"] = contexts
sft_df["response"] = responses

In [19]:
sft_df

,question,context,response
0,What must I learn before I can effectively stu...,"[[STUDENT REVIEW] Metadata:\n CRN: 31274, C...","Based on the student feedback, it seems that h..."
1,How does Professor Alikhani's grading system i...,[[COURSE INFO] === course metadata ===\ncourse...,"Based on the student review provided, Professo..."
2,What criteria does Professor Fuentes-Cabrera u...,[[COURSE INFO] === course metadata ===\ncourse...,"Based on the provided context, there is no spe..."
3,What are the regular class hours for the Mobil...,[[COURSE INFO] === course metadata ===\ncourse...,The regular class hours for the Mobile Applica...
4,"Could you elaborate on Professor Das, Maitraye...",[[COURSE INFO] === course metadata ===\ncourse...,"Based on the available reviews, Professor Mait..."
...,...,...,...
245,In what manner does Professor Aibin and Michal...,[[COURSE INFO] === course metadata ===\ncourse...,"Based on the course descriptions provided, Pro..."
246,Which academic tracks include courses focused ...,"[[STUDENT REVIEW] Metadata:\n CRN: 36913, C...","Based on the information provided, there isn't..."
247,Could you share with me the courses that Profe...,"[[STUDENT REVIEW] Metadata:\n CRN: 31172, C...","Based on the information provided, Professor T..."
248,Which educator is responsible for the Database...,"[[STUDENT REVIEW] Metadata:\n CRN: 39614, C...","Based on the reviews provided, there are multi..."


In [23]:
from datasets import Dataset

sft_dataset = Dataset.from_pandas(sft_df)

In [26]:
repo_id = "kishore-s-15/curriculum_compass_sft_dataset" 

sft_dataset.push_to_hub(repo_id)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]